# Import modules

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import cv2
import json
from PIL import Image
import shutil
from tqdm import tqdm
import torch

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

In [ ]:
!mkdir -p /root/.config/Ultralytics
!cp /kaggle/input/yolov5-font/Arial.ttf /root/.config/Ultralytics/

# Initialise environment

In [ ]:
# for submission format
import sys
sys.path.append('../input/tensorflow-great-barrier-reef')

In [ ]:
import greatbarrierreef

env = greatbarrierreef.make_env()   # initialize the environment
iter_test = env.iter_test()

# Train Model

In [ ]:
model = torch.hub.load('/kaggle/input/yolov5-lib-ds', 'custom', path='/kaggle/input/weights/best.pt', source='local', force_reload=True)  # or yolov5m, yolov5l, yolov5x, custom

# Submission environment

This section contains code to actually submit for the competition.

## Submit predictions

In [ ]:
# initialise variables
idc = 0
submission_dict = {
    'index': [],
    'annotations': [],
}

# submit predictions
for (image, sample_prediction_df) in iter_test:
    results = model(image)
    predictions = []
    prediction_str = ''
    
    submission_dict['index'].append(idc)
    for i in results.xyxy[0].tolist():
        x1, y1, x2, y2, _, _ = map(int, i)
        _, _, _, _, c,_ = i
        # if c < 0.2:
        #     continue
        w = x2 - x1
        h = y2 - y1
        predictions.append('{:.2f} {} {} {} {}'.format(c, x1, y1, w, h))
    
    prediction_str = ' '.join(predictions)
    submission_dict['annotations'].append(prediction_str)
    # sample_prediction_df['annotations'] = '0 0 0 0 0'
    sample_prediction_df['annotations'] = ''
    # sample_prediction_df['annotations'] = prediction_str
    
    for key,value in sample_prediction_df.items():
        print(key, ':', value)
    
    env.predict(sample_prediction_df)
    idc+=1

In [ ]:
sample_prediction_df.head()

## Create Submission

In [ ]:
df= pd.DataFrame(submission_dict)
df.to_csv("submission.csv", index = False, na_rep = '0')
df.head()

## Check Submission

In [ ]:
sub_df = pd.read_csv('submission.csv')
sub_df.head()

# Development
This section contains code for testing and development.

## Initialise variables and small test set

In [ ]:
idc = 0
submission_dict = {
    'index': [],
    'annotations': [],
}

# two images
image = ("/kaggle/input/tensorflow-great-barrier-reef/train_images/video_0/0.jpg","/kaggle/input/tensorflow-great-barrier-reef/train_images/video_0/63.jpg")

## Make predictions
Set `dev = True` to overwrite previous `submission.csv` file.

In [ ]:
dev = False
# dev = True

if dev:
    for path in image:
        results = model(path)
        predictions = []
        prediction_str = ''
        submission_dict['index'].append(idc)
        print(len(results.xyxy[0].tolist()))
        print(results.xyxy[0].tolist())

        # append all results together for multiple COTS
        for i in results.xyxy[0].tolist():
            x1, y1, x2, y2, _, _ = map(int, i)
            _, _, _, _, c,_ = i
            # if c < 0.2:
                # continue
            w = x2 - x1
            h = y2 - y1
            predictions.append('{:.2f} {:d} {:d} {:d} {:d}'.format(c, x1, y1, w, h))

            prediction_str = ' '.join(predictions)
        submission_dict['annotations'].append(prediction_str)
        idc+=1

In [ ]:
print(submission_dict)

df= pd.DataFrame(submission_dict)
df